In [132]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from RNNmodel import create_network
import matplotlib.pyplot as plt

In [133]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pkl.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [134]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 100
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_20 (Embedding)       (None, None, 100)    4800        ['input_21[0][0]']               
                                                                                                  
 embedding_21 (Embedding)       (None, None, 100)    1900        ['input_22[0][0]']               
                                                                                           

In [135]:
import pickle
piece = None
with open('../data/data_2/data_germany_1.pkl', 'rb') as handle:
    piece = pickle.load(handle)

In [136]:
notes_input = [ notes_to_int[note] for note in piece[0][0] ]
print(notes_input)

[16, 21, 21, 21, 21, 21, 23, 25, 25, 25, 28, 23, 46, 28, 21, 46, 16, 21, 21, 21, 21, 23, 25, 25, 25, 28, 23, 46, 28, 21, 46, 28, 33, 32, 30, 28, 26, 25, 46, 25, 25, 46, 30, 28, 26, 46, 25, 26, 46, 30, 28, 25, 46, 24, 25, 46, 28, 33, 32, 30, 28, 26, 25, 46, 24, 25, 46, 28, 28, 30, 30, 30, 32, 30, 32, 33, 33, 46]


In [137]:
durations_input = [ durations_to_int[duration] for duration in piece[0][1] ]
print(durations_input)

[7, 7, 7, 6, 2, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 7, 7, 7, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 4, 4, 13, 10, 13, 10, 10, 7, 7, 10, 10, 10, 10, 10, 7, 7, 10, 10, 10, 10, 10, 7, 7, 10, 10, 10, 10, 13, 10, 13, 10, 10, 7, 7, 10, 1, 7, 7, 12, 7, 10, 10, 10, 10, 14, 10, 10]


In [138]:
phrases_input = piece[0][2]
print(phrases_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [139]:
max_extra_notes = len(notes_input) - 1
seq_len = 5
phrases_output = []
phrases_output.append(0) # after the first note (that is already in the initial sequence)

notes_input_sequence = [notes_input[0]] # the first note element of the sequence
durations_input_sequence = [durations_input[0]] # the first duration element of the sequence

if seq_len is not None:
    notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
    durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations

In [140]:
print(notes_input_sequence)

[47, 47, 47, 47, 16]


In [141]:
print(durations_input_sequence)

[0, 0, 0, 0, 7]


In [142]:
for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    phrases_prediction = model.predict(prediction_input, verbose=0)
    print(notes_input_sequence)
    print(durations_input_sequence)
    print(phrases_prediction[0][0])
    print(phrases_prediction[0][1])
    print("===")
    print(phrases_prediction)
    
    phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

    notes_input_sequence.append(notes_input[note_index])
    durations_input_sequence.append(durations_input[note_index])
    
    if len(notes_input_sequence) > seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]

[47, 47, 47, 47, 16]
[0, 0, 0, 0, 7]
1.0
2.1530525e-17
===
[[1.0000000e+00 2.1530525e-17]]
[47, 47, 47, 16, 16]
[0, 0, 0, 7, 7]
1.0
2.6647688e-16
===
[[1.0000000e+00 2.6647688e-16]]
[47, 47, 16, 16, 21]
[0, 0, 7, 7, 7]
1.0
2.0324304e-15
===
[[1.0000000e+00 2.0324304e-15]]
[47, 16, 16, 21, 21]
[0, 7, 7, 7, 7]
1.0
4.313685e-14
===
[[1.000000e+00 4.313685e-14]]
[16, 16, 21, 21, 21]
[7, 7, 7, 7, 6]
0.9999999
1.16650696e-07
===
[[9.99999881e-01 1.16650696e-07]]
[16, 21, 21, 21, 21]
[7, 7, 7, 6, 2]
1.0
2.0845131e-11
===
[[1.0000000e+00 2.0845131e-11]]
[21, 21, 21, 21, 21]
[7, 7, 6, 2, 4]
1.0
1.6766905e-13
===
[[1.0000000e+00 1.6766905e-13]]
[21, 21, 21, 21, 23]
[7, 6, 2, 4, 4]
1.0
5.6302766e-12
===
[[1.0000000e+00 5.6302766e-12]]
[21, 21, 21, 23, 25]
[6, 2, 4, 4, 7]
1.0
3.5289936e-08
===
[[1.0000000e+00 3.5289936e-08]]
[21, 21, 23, 25, 25]
[2, 4, 4, 7, 7]
0.99999833
1.6308735e-06
===
[[9.9999833e-01 1.6308735e-06]]
[21, 23, 25, 25, 25]
[4, 4, 7, 7, 7]
0.99992657
7.3388386e-05
===
[[9.9992657

In [143]:
print(phrases_output) # unfortunately completely wrong...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [144]:
print(phrases_input) # should be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
